# RBRP Dry Protocol - バイオインフォマティクス解析自動化パイプライン

このノートブックは、Eric Koolラボの論文「Reactivity-based RNA profiling for analyzing transcriptome interactions of small molecules in human cells」のドライプロトコル部分を自動化します。

**対象ユーザー**: バイオインフォマティクス初学者  
**入力**: FASTQファイル（ペアエンド対応）  
**出力**: RNA-薬物相互作用解析結果

## 🔧 モダンシーケンサー対応
最近のシーケンサー（Illumina NovaSeq、NextSeq等）では以下の処理が自動で実行されるため、スキップオプションを用意しています：

- **デマルチプレックス**: バーコード分離済みFASTQファイルの出力
- **アダプタートリミング**: アダプター配列除去済みの出力
- **品質フィルタリング**: 低品質リード除去済みの出力

デフォルト設定では、これらの処理をスキップして高速化されています。

## 使用方法
1. 設定セクションで入力ファイルパスを指定
2. スキップオプションを必要に応じて調整
3. 「Run All Cells」で全自動実行
4. 結果は`data/results/`フォルダに保存されます

## 1. 環境設定と初期化

In [1]:
import os
import sys
import subprocess
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import yaml
import logging
from datetime import datetime
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# プロジェクトルートディレクトリを設定
PROJECT_ROOT = Path().resolve().parent
sys.path.append(str(PROJECT_ROOT / 'src'))

# ログ設定
log_file = PROJECT_ROOT / 'logs' / f'rbrp_pipeline_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log'
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

print(f"🧬 RBRP Dry Protocol Pipeline Started")
print(f"📁 Project Root: {PROJECT_ROOT}")
print(f"📝 Log File: {log_file}")

🧬 RBRP Dry Protocol Pipeline Started
📁 Project Root: /home/akahod3f/work/kool
📝 Log File: /home/akahod3f/work/kool/logs/rbrp_pipeline_20250917_044525.log


## 2. 設定ファイル読み込み・入力ファイル指定

In [ ]:
# ===== ユーザー設定 =====
# ここで入力ファイルを指定してください

# 入力FASTQファイルのパス（ペアエンド対応）
# Kool論文のSRA accession numbers (GSE229331) をデフォルトに使用
# download_rbrp_data.pyでダウンロードした場合のデフォルトパス（data/raw）
INPUT_FASTQ_FILES = {
    'SRR22397001': {  # HEK293_Probe2_only_rep1
        'R1': 'data/raw/SRR22397001_1.fastq',
        'R2': 'data/raw/SRR22397001_2.fastq'
    },
    'SRR22397002': {  # HEK293_Probe2_only_rep2
        'R1': 'data/raw/SRR22397002_1.fastq',
        'R2': 'data/raw/SRR22397002_2.fastq'
    },
    'SRR22397003': {  # HEK293_Probe2_Levofloxacin_rep1
        'R1': 'data/raw/SRR22397003_1.fastq',
        'R2': 'data/raw/SRR22397003_2.fastq'
    },
    'SRR22397004': {  # HEK293_Probe2_Levofloxacin_rep2
        'R1': 'data/raw/SRR22397004_1.fastq',
        'R2': 'data/raw/SRR22397004_2.fastq'
    },
    'SRR22397005': {  # HEK293_DMSO_ctrl_rep1
        'R1': 'data/raw/SRR22397005_1.fastq',
        'R2': 'data/raw/SRR22397005_2.fastq'
    },
    'SRR22397006': {  # HEK293_DMSO_ctrl_rep2
        'R1': 'data/raw/SRR22397006_1.fastq',
        'R2': 'data/raw/SRR22397006_2.fastq'
    },
    'SRR22397007': {  # HEK293_Levofloxacin_ctrl_rep1
        'R1': 'data/raw/SRR22397007_1.fastq',
        'R2': 'data/raw/SRR22397007_2.fastq'
    },
    'SRR22397008': {  # HEK293_Levofloxacin_ctrl_rep2
        'R1': 'data/raw/SRR22397008_1.fastq',
        'R2': 'data/raw/SRR22397008_2.fastq'
    }
}

# 参照ゲノムファイル
REFERENCE_FILES = {
    'genome_fasta': '/path/to/genome.fa',
    'genome_gtf': '/path/to/genome.gtf',
    'transcriptome_index': '/path/to/transcriptome_index',
    'adapter_fasta': '/path/to/adapter.fa'
}

# バーコード情報（demultiplexをスキップしない場合のみ使用）
BARCODES = {
    'SRR22397001': 'GGTT',  # HEK293_Probe2_only_rep1
    'SRR22397002': 'TTGT',  # HEK293_Probe2_only_rep2
    'SRR22397003': 'ACCT',  # HEK293_Probe2_Levofloxacin_rep1
    'SRR22397004': 'CAAT',  # HEK293_Probe2_Levofloxacin_rep2
    'SRR22397005': 'GCAA',  # HEK293_DMSO_ctrl_rep1
    'SRR22397006': 'AATC',  # HEK293_DMSO_ctrl_rep2
    'SRR22397007': 'TGAC',  # HEK293_Levofloxacin_ctrl_rep1
    'SRR22397008': 'CGGT'   # HEK293_Levofloxacin_ctrl_rep2
}

# 解析パラメータ
ANALYSIS_PARAMS = {
    'min_read_length': 25,
    'min_rpkm': 1,
    'min_sequencing_depth': 200,
    'rbrp_score_threshold': 0.12,
    'p_value_threshold': 0.05
}

# 🔧 処理スキップオプション
PROCESSING_OPTIONS = {
    'skip_demultiplex': True,            # True: デマルチプレックスをスキップ
    'skip_adapter_trimming': True,       # True: アダプタートリミングをスキップ
    'skip_pcr_duplicate_removal': False, # True: PCR重複除去をスキップ
    'perform_quality_control': False,    # True: FastQCによる品質確認を実行
    'adapter_sequences': {               # カスタムアダプター配列（trimming実行時のみ使用）
        'R1': 'AGATCGGAAGAGCGGTTCAG',
        'R2': 'AGATCGGAAGAGCGGTTCAG'
    }
}

print("✅ 設定完了（SRA accession numbers使用・data/rawディレクトリ・ペアエンド対応・スキップオプション付き）")
print(f"📊 解析対象サンプル数: {len(INPUT_FASTQ_FILES)}")
print(f"📁 入力ディレクトリ: data/raw (download_rbrp_data.pyのデフォルト)")
print(f"🔧 デマルチプレックススキップ: {PROCESSING_OPTIONS['skip_demultiplex']}")
print(f"🔧 アダプタートリミングスキップ: {PROCESSING_OPTIONS['skip_adapter_trimming']}")
logger.info(f"Analysis started with {len(INPUT_FASTQ_FILES)} paired-end samples from Kool lab (GSE229331) in data/raw")

## 3. 依存関係チェック・外部ツール確認

In [ ]:
def check_external_tools():
    """外部ツールの存在確認"""
    required_tools = [
        'fastqc',
        'bowtie2',
        'gffread',
        'wiggletools',
        'bedGraphToBigWig',
        'wigToBigWig'
    ]
    
    missing_tools = []
    
    for tool in required_tools:
        try:
            result = subprocess.run(['which', tool], capture_output=True, text=True)
            if result.returncode == 0:
                print(f"✅ {tool}: {result.stdout.strip()}")
            else:
                missing_tools.append(tool)
                print(f"❌ {tool}: 見つかりません")
        except Exception as e:
            missing_tools.append(tool)
            print(f"❌ {tool}: エラー - {e}")
    
    if missing_tools:
        print(f"\n⚠️ 以下のツールが不足しています: {', '.join(missing_tools)}")
        print("インストール方法はREADME.mdを参照してください")
        return False
    else:
        print("\n🎉 すべての必要ツールが利用可能です")
        return True

tools_available = check_external_tools()

## 4. パイプライン実行関数群

In [ ]:
def run_command(cmd, description, check=True):
    """コマンド実行のヘルパー関数"""
    logger.info(f"実行中: {description}")
    logger.debug(f"コマンド: {cmd}")
    
    try:
        result = subprocess.run(cmd, shell=True, check=check, 
                              capture_output=True, text=True)
        if result.returncode == 0:
            logger.info(f"✅ 完了: {description}")
            return result
        else:
            logger.error(f"❌ 失敗: {description}")
            logger.error(f"エラー出力: {result.stderr}")
            if check:
                raise subprocess.CalledProcessError(result.returncode, cmd)
            return result
    except Exception as e:
        logger.error(f"❌ 例外発生: {description} - {e}")
        if check:
            raise
        return None

def find_latest_processed_files(sample_name, file_type="fastq"):
    """
    サンプルの最新処理済みファイルを動的に検索
    
    Args:
        sample_name (str): サンプル名
        file_type (str): ファイルタイプ ("fastq", "sam", "rt" など)
    
    Returns:
        dict: {"R1": path, "R2": path} または None
    """
    search_patterns = {
        "fastq": [
            f"data/processed/trimmed/{sample_name}_trimmed",  # トリミング済み
            f"data/processed/{sample_name}_rmdup",            # PCR重複除去済み
            f"data/processed/{sample_name}_demux",            # デマルチプレックス済み
        ],
        "sam": [
            f"data/processed/aligned/{sample_name}.sam"
        ],
        "rt": [
            f"data/processed/aligned/{sample_name}.rt"
        ]
    }
    
    if file_type == "fastq":
        # ペアエンドファイルの場合
        for pattern in search_patterns[file_type]:
            r1_file = PROJECT_ROOT / f"{pattern}_1.fastq"
            r2_file = PROJECT_ROOT / f"{pattern}_2.fastq"
            
            if r1_file.exists() and r2_file.exists():
                logger.info(f"✅ {sample_name}: 使用ファイル {pattern}_*.fastq")
                return {"R1": r1_file, "R2": r2_file}
        
        # 最後の手段：元のFASTQファイル
        original_r1 = INPUT_FASTQ_FILES[sample_name]['R1']
        original_r2 = INPUT_FASTQ_FILES[sample_name]['R2']
        if Path(original_r1).exists() and Path(original_r2).exists():
            logger.warning(f"⚠️ {sample_name}: 元ファイルを使用 {original_r1}, {original_r2}")
            return {"R1": Path(original_r1), "R2": Path(original_r2)}
            
        return None
    
    else:
        # 単一ファイルの場合
        for pattern in search_patterns.get(file_type, []):
            file_path = PROJECT_ROOT / pattern
            if file_path.exists():
                logger.info(f"✅ {sample_name}: 使用ファイル {pattern}")
                return file_path
        return None

def create_output_dirs():
    """出力ディレクトリ作成"""
    dirs = [
        'logs',  # ログディレクトリを追加
        'data/processed/fastqc',
        'data/processed/trimmed', 
        'data/processed/aligned',
        'data/processed/rbrp_scores',
        'data/results/bigwig',
        'data/results/figures'
    ]
    
    for dir_path in dirs:
        (PROJECT_ROOT / dir_path).mkdir(parents=True, exist_ok=True)
    
    logger.info("📁 出力ディレクトリを作成しました")

create_output_dirs()
print("📁 出力ディレクトリ準備完了")

## 5. ステップ1: 品質管理・デマルチプレックス

In [ ]:
def step1_quality_control_and_demultiplex():
    """ステップ1: 品質管理とデマルチプレックス（スキップオプション対応）"""
    print("\n🔍 ステップ1: 品質管理・デマルチプレックス開始")
    
    # スキップオプションの確認
    skip_demux = PROCESSING_OPTIONS.get('skip_demultiplex', False)
    perform_qc = PROCESSING_OPTIONS.get('perform_quality_control', True)
    
    if skip_demux:
        print("🚀 デマルチプレックスをスキップ（最近のシーケンサーでは既に実行済み）")
    if not perform_qc:
        print("🚀 品質管理をスキップ")
    
    for sample_name, fastq_paths in tqdm(INPUT_FASTQ_FILES.items(), desc="Processing samples"):
        # ペアエンドファイルの存在確認
        r1_path = fastq_paths['R1']
        r2_path = fastq_paths['R2']
        
        if not Path(r1_path).exists():
            logger.warning(f"⚠️ R1ファイルが見つかりません: {r1_path}")
            continue
        if not Path(r2_path).exists():
            logger.warning(f"⚠️ R2ファイルが見つかりません: {r2_path}")
            continue
        
        # FastQC実行（オプション）
        if perform_qc:
            fastqc_cmd_r1 = f"fastqc -o {PROJECT_ROOT}/data/processed/fastqc {r1_path}"
            fastqc_cmd_r2 = f"fastqc -o {PROJECT_ROOT}/data/processed/fastqc {r2_path}"
            
            run_command(fastqc_cmd_r1, f"FastQC for {sample_name} R1")
            run_command(fastqc_cmd_r2, f"FastQC for {sample_name} R2")
        else:
            print(f"⏭️ {sample_name}: FastQCをスキップしました")
        
        # デマルチプレックス（オプション）
        if not skip_demux and sample_name in BARCODES:
            barcode = BARCODES[sample_name]
            output_r1 = PROJECT_ROOT / f"data/processed/{sample_name}_demux_1.fastq"
            output_r2 = PROJECT_ROOT / f"data/processed/{sample_name}_demux_2.fastq"
            
            # ペアエンドバーコード抽出
            demux_cmd_r1 = f"""grep -A 3 "^@.*{barcode}" {r1_path} | grep -v "^--$" > {output_r1}"""
            demux_cmd_r2 = f"""grep -A 3 "^@.*{barcode}" {r2_path} | grep -v "^--$" > {output_r2}"""
            
            run_command(demux_cmd_r1, f"Demultiplexing {sample_name} R1", check=False)
            run_command(demux_cmd_r2, f"Demultiplexing {sample_name} R2", check=False)
            
            logger.info(f"✅ {sample_name}: ペアエンドデマルチプレックス完了")
        elif skip_demux:
            # デマルチプレックスをスキップする場合、元ファイルへのシンボリックリンクを作成
            output_r1 = PROJECT_ROOT / f"data/processed/{sample_name}_demux_1.fastq"
            output_r2 = PROJECT_ROOT / f"data/processed/{sample_name}_demux_2.fastq"
            
            # シンボリックリンク作成（既に存在する場合は削除）
            if output_r1.exists():
                output_r1.unlink()
            if output_r2.exists():
                output_r2.unlink()
                
            output_r1.symlink_to(Path(r1_path).absolute())
            output_r2.symlink_to(Path(r2_path).absolute())
            
            logger.info(f"✅ {sample_name}: デマルチプレックスをスキップ（シンボリックリンク作成）")
        else:
            logger.warning(f"⚠️ {sample_name}: バーコードが指定されていません")
    
    skip_msg = []
    if skip_demux:
        skip_msg.append("デマルチプレックス")
    if not perform_qc:
        skip_msg.append("品質管理")
    
    completion_msg = "✅ ステップ1完了: 品質管理・デマルチプレックス"
    if skip_msg:
        completion_msg += f"（{', '.join(skip_msg)}をスキップ）"
    
    print(completion_msg)

step1_quality_control_and_demultiplex()

## 6. ステップ2: PCR重複除去・トリミング

In [ ]:
def step2_remove_duplicates_and_trim():
    """ステップ2: PCR重複除去とトリミング（動的ファイル解決対応）"""
    print("\n✂️ ステップ2: PCR重複除去・トリミング開始（動的ファイル解決対応）")
    
    # スキップオプションの確認
    skip_trimming = PROCESSING_OPTIONS.get('skip_adapter_trimming', False)
    skip_pcr_removal = PROCESSING_OPTIONS.get('skip_pcr_duplicate_removal', False)
    
    if skip_trimming:
        print("🚀 アダプタートリミングをスキップ（最近のシーケンサーでは既に実行済み）")
    if skip_pcr_removal:
        print("🚀 PCR重複除去をスキップ")
    
    for sample_name in tqdm(INPUT_FASTQ_FILES.keys(), desc="Processing trimming"):
        # 前ステップのファイルを動的に検索（デマルチプレックス済み）
        input_r1 = PROJECT_ROOT / f"data/processed/{sample_name}_demux_1.fastq"
        input_r2 = PROJECT_ROOT / f"data/processed/{sample_name}_demux_2.fastq"
        
        if not input_r1.exists() or not input_r2.exists():
            logger.warning(f"⚠️ {sample_name}: デマルチプレックス済みファイルが見つかりません")
            # 元ファイルの確認
            original_r1 = Path(INPUT_FASTQ_FILES[sample_name]['R1'])
            original_r2 = Path(INPUT_FASTQ_FILES[sample_name]['R2'])
            if original_r1.exists() and original_r2.exists():
                input_r1, input_r2 = original_r1, original_r2
                logger.info(f"📄 {sample_name}: 元ファイルを使用")
            else:
                logger.error(f"❌ {sample_name}: 入力ファイルが見つかりません")
                continue
        
        # PCR重複除去（オプション）
        if not skip_pcr_removal:
            rmdup_r1 = PROJECT_ROOT / f"data/processed/{sample_name}_rmdup_1.fastq"
            rmdup_r2 = PROJECT_ROOT / f"data/processed/{sample_name}_rmdup_2.fastq"
            
            # ペアエンドPCR重複除去（簡易版）
            rmdup_cmd_r1 = f"""awk '/^@/ {{if (seen[$0]++) next}} 1' {input_r1} > {rmdup_r1}"""
            rmdup_cmd_r2 = f"""awk '/^@/ {{if (seen[$0]++) next}} 1' {input_r2} > {rmdup_r2}"""
            
            run_command(rmdup_cmd_r1, f"Remove duplicates for {sample_name} R1")
            run_command(rmdup_cmd_r2, f"Remove duplicates for {sample_name} R2")
            
            # 次のステップの入力ファイルを更新
            trim_input_r1 = rmdup_r1
            trim_input_r2 = rmdup_r2
        else:
            print(f"⏭️ {sample_name}: PCR重複除去をスキップしました")
            trim_input_r1 = input_r1
            trim_input_r2 = input_r2
        
        # アダプタートリミング（オプション）
        trimmed_r1 = PROJECT_ROOT / f"data/processed/trimmed/{sample_name}_trimmed_1.fastq"
        trimmed_r2 = PROJECT_ROOT / f"data/processed/trimmed/{sample_name}_trimmed_2.fastq"
        
        if not skip_trimming:
            # ペアエンドアダプター除去とクオリティトリミング
            adapter_r1 = PROCESSING_OPTIONS['adapter_sequences']['R1']
            adapter_r2 = PROCESSING_OPTIONS['adapter_sequences']['R2']
            
            trim_cmd = f"""cutadapt -a {adapter_r1} -A {adapter_r2} \
                          -q 30 -m {ANALYSIS_PARAMS['min_read_length']} \
                          -o {trimmed_r1} -p {trimmed_r2} \
                          {trim_input_r1} {trim_input_r2}"""
            
            run_command(trim_cmd, f"Paired-end adapter trimming for {sample_name}", check=False)
        else:
            # トリミングをスキップする場合、シンボリックリンクを作成
            if trimmed_r1.exists():
                trimmed_r1.unlink()
            if trimmed_r2.exists():
                trimmed_r2.unlink()
                
            trimmed_r1.symlink_to(trim_input_r1.absolute())
            trimmed_r2.symlink_to(trim_input_r2.absolute())
            
            print(f"⏭️ {sample_name}: アダプタートリミングをスキップ（シンボリックリンク作成）")
        
        logger.info(f"✅ {sample_name}: 処理完了")
    
    # 完了メッセージ作成
    skip_msg = []
    if skip_trimming:
        skip_msg.append("アダプタートリミング")
    if skip_pcr_removal:
        skip_msg.append("PCR重複除去")
    
    completion_msg = "✅ ステップ2完了: PCR重複除去・トリミング（動的ファイル解決対応）"
    if skip_msg:
        completion_msg += f"（{', '.join(skip_msg)}をスキップ）"
    
    print(completion_msg)

step2_remove_duplicates_and_trim()

## 7. ステップ3: 配列アライメント・転写産物発現量計算

In [ ]:
def step3_alignment_and_expression():
    """ステップ3: 配列アライメントと転写産物発現量計算（動的ファイル解決対応）"""
    print("\n🧬 ステップ3: アライメント・発現量計算開始（動的ファイル解決対応）")
    
    for sample_name in tqdm(INPUT_FASTQ_FILES.keys(), desc="Processing alignment"):
        # 動的に最新の処理済みFASTQファイルを検索
        fastq_files = find_latest_processed_files(sample_name, "fastq")
        
        if not fastq_files:
            logger.error(f"❌ {sample_name}: 処理済みFASTQファイルが見つかりません")
            continue
            
        trimmed_r1 = fastq_files["R1"]
        trimmed_r2 = fastq_files["R2"]
        
        if not trimmed_r1.exists():
            logger.warning(f"⚠️ R1ファイルが見つかりません: {trimmed_r1}")
            continue
        if not trimmed_r2.exists():
            logger.warning(f"⚠️ R2ファイルが見つかりません: {trimmed_r2}")
            continue
        
        # Bowtie2でペアエンドアライメント
        sam_file = PROJECT_ROOT / f"data/processed/aligned/{sample_name}.sam"
        
        # ペアエンドマッピング（-1, -2でペアエンドファイル指定）
        align_cmd = f"""bowtie2 -1 {trimmed_r1} -2 {trimmed_r2} -S {sam_file} \
                       -x {REFERENCE_FILES['transcriptome_index']} \
                       --non-deterministic --time \
                       --minins 50 --maxins 500 \
                       --no-mixed --no-discordant"""
        
        run_command(align_cmd, f"Paired-end alignment for {sample_name}")
        
        # RPKM計算
        rpkm_file = PROJECT_ROOT / f"data/processed/aligned/{sample_name}.rpkm"
        rpkm_cmd = f"""python {PROJECT_ROOT}/scripts/calculate_rpkm.py -i {sam_file} -o {rpkm_file}"""
        
        run_command(rpkm_cmd, f"RPKM calculation for {sample_name}", check=False)
        
        # RTstop計算
        rt_file = PROJECT_ROOT / f"data/processed/aligned/{sample_name}.rt"
        rt_cmd = f"""python {PROJECT_ROOT}/scripts/calculate_rtstops.py \
                    -i {sam_file} -o {rt_file} -r {rpkm_file} -c {ANALYSIS_PARAMS['min_rpkm']}"""
        
        run_command(rt_cmd, f"RTstop calculation for {sample_name}", check=False)
        logger.info(f"✅ {sample_name}: ペアエンドアライメント・発現量計算完了")
    
    print("✅ ステップ3完了: アライメント・発現量計算（動的ファイル解決対応）")

step3_alignment_and_expression()

## 8. ステップ4: RBRPスコア計算・統計解析

In [ ]:
def step4_rbrp_score_calculation():
    """ステップ4: RBRPスコア計算と統計解析（動的ファイル解決対応）"""
    print("\n📊 ステップ4: RBRPスコア計算・統計解析開始（動的ファイル解決対応）")
    
    # SRA accession numbersに基づくサンプルグループ分け
    # GSE229331のKool論文データ構成に基づく
    probe_samples = [
        'SRR22397001',  # HEK293_Probe2_only_rep1
        'SRR22397002',  # HEK293_Probe2_only_rep2
        'SRR22397003',  # HEK293_Probe2_Levofloxacin_rep1
        'SRR22397004'   # HEK293_Probe2_Levofloxacin_rep2
    ]
    
    control_samples = [
        'SRR22397005',  # HEK293_DMSO_ctrl_rep1
        'SRR22397006',  # HEK293_DMSO_ctrl_rep2
        'SRR22397007',  # HEK293_Levofloxacin_ctrl_rep1
        'SRR22397008'   # HEK293_Levofloxacin_ctrl_rep2
    ]
    
    print(f"📋 プローブサンプル: {probe_samples}")
    print(f"📋 コントロールサンプル: {control_samples}")
    
    # バックグラウンドRTstopファイルをマージ
    if len(control_samples) >= 2:
        control_files = []
        for sample in control_samples:
            rt_file = find_latest_processed_files(sample, "rt")
            if rt_file and rt_file.exists():
                control_files.append(str(rt_file))
            else:
                logger.warning(f"⚠️ {sample}: RTファイルが見つかりません")
        
        if control_files:
            merged_control = PROJECT_ROOT / "data/processed/rbrp_scores/merged_control.rt"
            merge_cmd = f"""python {PROJECT_ROOT}/scripts/merge_rt_files.py \
                           -i {':'.join(control_files)} -o {merged_control}"""
            run_command(merge_cmd, "Merging control RTstop files", check=False)
    
    # 各プローブサンプルのRBRPスコア計算
    for sample_name in tqdm(probe_samples, desc="Calculating RBRP scores"):
        rt_file = find_latest_processed_files(sample_name, "rt")
        
        if not rt_file or not rt_file.exists():
            logger.warning(f"⚠️ {sample_name}: RTファイルが見つかりません")
            continue
        
        # RTファイル正規化
        normalized_file = PROJECT_ROOT / f"data/processed/rbrp_scores/{sample_name}_normalized.rt"
        norm_cmd = f"""python {PROJECT_ROOT}/scripts/normalize_rt_file.py \
                      -i {rt_file} -o {normalized_file} -d 32 -l 32"""
        run_command(norm_cmd, f"Normalizing RT file for {sample_name}", check=False)
        
        # RBRPスコア計算
        rbrp_file = PROJECT_ROOT / f"data/processed/rbrp_scores/{sample_name}_rbrp.out"
        background_file = merged_control if 'merged_control' in locals() else None
        
        if background_file and background_file.exists():
            rbrp_cmd = f"""python {PROJECT_ROOT}/scripts/calculate_rbrp_score.py \
                          -f {normalized_file} -b {background_file} -o {rbrp_file} \
                          -e dividing -y 0.5"""
        else:
            rbrp_cmd = f"""python {PROJECT_ROOT}/scripts/calculate_rbrp_score.py \
                          -f {normalized_file} -o {rbrp_file}"""
        
        run_command(rbrp_cmd, f"Calculating RBRP scores for {sample_name}", check=False)
        
        # 低品質スコアフィルタリング
        filtered_file = PROJECT_ROOT / f"data/processed/rbrp_scores/{sample_name}_filtered.out"
        filter_cmd = f"""python {PROJECT_ROOT}/scripts/filter_rbrp_scores.py \
                        -i {rbrp_file} -o {filtered_file} \
                        -t {ANALYSIS_PARAMS['min_sequencing_depth']} -s 5 -e 30"""
        run_command(filter_cmd, f"Filtering RBRP scores for {sample_name}", check=False)
        
        logger.info(f"✅ {sample_name}: RBRPスコア計算完了")
    
    print("✅ ステップ4完了: RBRPスコア計算・統計解析（動的ファイル解決対応）")

step4_rbrp_score_calculation()

## 9. ステップ5: 可視化ファイル生成・結果出力

In [ ]:
def step5_visualization_and_output():
    """ステップ5: 可視化ファイル生成と結果出力"""
    print("\n📈 ステップ5: 可視化ファイル生成・結果出力開始")
    
    # プローブサンプルを定義（SRA accession numbers基準）
    probe_samples = [
        'SRR22397001',  # HEK293_Probe2_only_rep1
        'SRR22397002',  # HEK293_Probe2_only_rep2
        'SRR22397003',  # HEK293_Probe2_Levofloxacin_rep1
        'SRR22397004'   # HEK293_Probe2_Levofloxacin_rep2
    ]
    
    for sample_name in tqdm(probe_samples, desc="Generating visualization files"):
        filtered_file = PROJECT_ROOT / f"data/processed/rbrp_scores/{sample_name}_filtered.out"
        
        if not filtered_file.exists():
            logger.warning(f"⚠️ フィルタリング済みファイルが見つかりません: {filtered_file}")
            continue
        
        # bedgraphファイル生成
        bedgraph_file = PROJECT_ROOT / f"data/results/bigwig/{sample_name}.bedgraph"
        bedgraph_cmd = f"""python {PROJECT_ROOT}/scripts/generate_bedgraph.py \
                          -i {filtered_file} -o {bedgraph_file} \
                          -g {REFERENCE_FILES['genome_gtf']} \
                          -a {REFERENCE_FILES.get('transcriptome_fasta', '')}"""
        run_command(bedgraph_cmd, f"Generating bedgraph for {sample_name}", check=False)
        
        # bigwigファイル生成（UCscツールが利用可能な場合）
        bigwig_file = PROJECT_ROOT / f"data/results/bigwig/{sample_name}.bw"
        genome_size_file = PROJECT_ROOT / "data/genome.size"  # 事前に準備が必要
        
        if bedgraph_file.exists():
            # ソートと重複除去
            sorted_bedgraph = PROJECT_ROOT / f"data/results/bigwig/{sample_name}_sorted.bedgraph"
            sort_cmd = f"sort -k1,1 -k2,3n {bedgraph_file} | uniq > {sorted_bedgraph}"
            run_command(sort_cmd, f"Sorting bedgraph for {sample_name}", check=False)
            
            # bigwig変換
            if genome_size_file.exists():
                bw_cmd = f"bedGraphToBigWig {sorted_bedgraph} {genome_size_file} {bigwig_file}"
                run_command(bw_cmd, f"Converting to bigwig for {sample_name}", check=False)
        
        logger.info(f"✅ {sample_name}: 可視化ファイル生成完了")
    
    print("✅ ステップ5完了: 可視化ファイル生成・結果出力")

step5_visualization_and_output()

## 10. 結果サマリー・統計情報

In [ ]:
def generate_summary_report():
    """結果サマリーレポート生成（動的ファイル解決対応）"""
    print("\n📋 結果サマリーレポート生成（動的ファイル解決対応）")
    
    summary_data = []
    
    for sample_name in INPUT_FASTQ_FILES.keys():
        sample_summary = {'sample_name': sample_name}
        
        # 動的ファイル検索結果を記録
        fastq_files = find_latest_processed_files(sample_name, "fastq")
        sam_file = find_latest_processed_files(sample_name, "sam")
        rt_file = find_latest_processed_files(sample_name, "rt")
        
        # ファイル存在確認
        sample_summary['input_files_exist'] = bool(fastq_files)
        sample_summary['aligned_exists'] = bool(sam_file and sam_file.exists())
        sample_summary['rt_analysis_exists'] = bool(rt_file and rt_file.exists())
        
        # RBRP スコアファイル確認
        rbrp_file = PROJECT_ROOT / f"data/processed/rbrp_scores/{sample_name}_filtered.out"
        sample_summary['rbrp_scores_exists'] = rbrp_file.exists()
        
        # ファイルサイズ情報
        if fastq_files:
            r1_size = fastq_files["R1"].stat().st_size / (1024**2) if fastq_files["R1"].exists() else 0
            r2_size = fastq_files["R2"].stat().st_size / (1024**2) if fastq_files["R2"].exists() else 0
            sample_summary['input_size_mb'] = round(r1_size + r2_size, 2)
            sample_summary['processing_method'] = str(fastq_files["R1"]).split('/')[-1].split('_')[1]  # demux, rmdup, trimmed
        else:
            sample_summary['input_size_mb'] = 0
            sample_summary['processing_method'] = 'none'
            
        if sam_file and sam_file.exists():
            sample_summary['aligned_size_mb'] = round(sam_file.stat().st_size / (1024**2), 2)
        else:
            sample_summary['aligned_size_mb'] = 0
            
        if rbrp_file.exists():
            sample_summary['rbrp_size_mb'] = round(rbrp_file.stat().st_size / (1024**2), 2)
        else:
            sample_summary['rbrp_size_mb'] = 0
        
        # パイプライン完成度
        completed_steps = sum([
            sample_summary['input_files_exist'],
            sample_summary['aligned_exists'], 
            sample_summary['rt_analysis_exists'],
            sample_summary['rbrp_scores_exists']
        ])
        sample_summary['pipeline_completion'] = f"{completed_steps}/4"
        sample_summary['pipeline_success'] = completed_steps == 4
        
        summary_data.append(sample_summary)
    
    # サマリーDataFrame作成
    summary_df = pd.DataFrame(summary_data)
    
    # 結果表示
    print("\n📊 処理結果サマリー（動的ファイル解決対応）:")
    print(summary_df.to_string(index=False))
    
    # CSVファイルとして保存
    summary_file = PROJECT_ROOT / "data/results/processing_summary_dynamic.csv"
    summary_df.to_csv(summary_file, index=False)
    
    # 統計情報
    successful_samples = summary_df['pipeline_success'].sum()
    total_samples = len(summary_df)
    
    # 処理方法別の統計
    processing_methods = summary_df['processing_method'].value_counts()
    
    print(f"\n📈 処理統計（動的ファイル解決対応）:")
    print(f"   総サンプル数: {total_samples}")
    print(f"   パイプライン完全成功: {successful_samples}")
    print(f"   成功率: {successful_samples/total_samples*100:.1f}%")
    print(f"   平均ファイルサイズ: {summary_df['input_size_mb'].mean():.1f} MB")
    print(f"   使用された処理方法:")
    for method, count in processing_methods.items():
        print(f"     - {method}: {count} サンプル")
    print(f"   サマリーファイル: {summary_file}")
    
    return summary_df

summary_report = generate_summary_report()

# 実行時間の記録
print(f"\n🎉 動的ファイル解決対応パイプライン実行完了!")
print(f"📁 結果ファイルは以下に保存されました:")
print(f"   - 処理済みデータ: {PROJECT_ROOT}/data/processed/")
print(f"   - 最終結果: {PROJECT_ROOT}/data/results/")
print(f"   - ログファイル: {log_file}")

logger.info("RBRP Dry Protocol Pipeline (Dynamic File Resolution) completed successfully")

## 11. 結果可視化（オプション）

In [ ]:
def create_visualization_plots():
    """結果の可視化プロット作成"""
    print("\n📊 結果可視化プロット作成")
    
    # 処理サマリーの可視化
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('RBRP Dry Protocol - 処理結果サマリー', fontsize=16, y=0.98)
    
    # 1. ファイルサイズ分布
    size_columns = [col for col in summary_report.columns if 'size_mb' in col]
    if size_columns:
        size_data = summary_report[size_columns].fillna(0)
        axes[0, 0].bar(range(len(size_columns)), size_data.mean(), 
                      tick_label=[col.replace('_size_mb', '') for col in size_columns])
        axes[0, 0].set_title('平均ファイルサイズ (MB)')
        axes[0, 0].tick_params(axis='x', rotation=45)
    
    # 2. 処理成功率
    success_columns = [col for col in summary_report.columns if 'exists' in col]
    if success_columns:
        success_rates = summary_report[success_columns].mean() * 100
        axes[0, 1].bar(range(len(success_rates)), success_rates.values,
                      tick_label=[col.replace('_exists', '') for col in success_columns])
        axes[0, 1].set_title('処理成功率 (%)')
        axes[0, 1].set_ylim(0, 100)
        axes[0, 1].tick_params(axis='x', rotation=45)
    
    # 3. サンプル別処理状況
    if success_columns:
        sample_success = summary_report[success_columns].sum(axis=1)
        axes[1, 0].bar(range(len(sample_success)), sample_success.values,
                      tick_label=summary_report['sample_name'])
        axes[1, 0].set_title('サンプル別完了ステップ数')
        axes[1, 0].tick_params(axis='x', rotation=45)
    
    # 4. 処理時間の目安（仮想データ）
    processing_steps = ['デマルチプレックス', 'トリミング', 'アライメント', 'RBRP計算', '可視化']
    estimated_times = [5, 10, 30, 20, 5]  # 分単位
    axes[1, 1].bar(processing_steps, estimated_times)
    axes[1, 1].set_title('ステップ別推定処理時間 (分)')
    axes[1, 1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    
    # 図を保存
    plot_file = PROJECT_ROOT / "data/results/figures/processing_summary.png"
    plt.savefig(plot_file, dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"📊 可視化プロット保存: {plot_file}")

# 可視化実行
try:
    create_visualization_plots()
except Exception as e:
    logger.warning(f"可視化プロット作成エラー: {e}")
    print("⚠️ 可視化プロットの作成でエラーが発生しましたが、パイプライン自体は正常に完了しています")